In [2]:
import pandas as pd
import random
import time
import pymongo
from pymongo import MongoClient
from parsel import Selector
from datetime import datetime
from time import sleep
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

# Connecting ot db

connection = MongoClient("mongodb://localhost:27017")
db = connection["IndeedJobPostDB"]

chrome_options = Options()

PROXY = "" # HOST:PORT

chrome_options = uc.ChromeOptions()
chrome_options.add_argument(f'--proxy-server={PROXY}')

driver = uc.Chrome(executable_path=r'chromedriver', options=chrome_options)

position = input('Enter your preferred skill: ').strip()
place = input('Enter the location: ').strip()
no_of_pages = int(input('Enter the number of pages to scrape: '))

Enter your preferred skill: Restaurant
Enter the location: New York, NY
Enter the number of pages to scrape: 100


In [3]:
job_data = []
for page in range(0, no_of_pages*10, 10):
    driver.get(f'https://www.indeed.com/jobs?q={position}&l={place}&sort=date&start={page}')
    time.sleep(random.uniform(8.5, 10.9))

    page_source = driver.page_source

    soup = BeautifulSoup(page_source,features="html.parser")
    divs = soup.find_all('div', 'slider_container css-77eoo7 eu4oa1w0')
    for item in divs:
        # Getting the Job Title
        title = item.find('a').text.strip()
        #print(title)
        
        # Getting the job type
        try:
            job_type = item.find('div', 'heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly')
            #print(job_type.text.strip())
            if "+1" in job_type:
                job_type = "Full-time, Part-time";
            rows =[i.get_text(strip=True) for i  in job_type.find_all('div', 'metadata') if i.get_text(strip=True)!=""]
            print(rows)
            salary = rows[0].strip()
            if len(rows) > 1:
                type_of_job = rows[1]
        except:
            try:
                salary = item.find('div', 'attribute_snippet').text.strip()
                type_of_job = ''
            except:
                salary = ''

        #print(rows)
        #print(rows[0])
        #print(rows[1])
        #Getting the Company Name
        try:
            company = item.find('span', 'companyName').text.strip()
        except:
            company = ''
        #print(company)
        
        #print(salary)
        #Getting the Location
        try:
            location = item.find('div', 'companyLocation').text.strip()
        except:
            location = ''
        #print(location)    
        #Getting the link
        try:
            link = 'https://www.indeed.com/' + item.h2.a.get('href')
        except:
            link = ''
        #print(link)
        
        # Date Posted
        try:
            date_posted = item.find('span', 'date').text.strip()
        except:
            date_posted = ''
        #print(date_posted)
        # Current Date
        try: 
            current_date = datetime.today().strftime('%y-%m-%d')
        except:
            current_date = ''
        #print(current_date)
        
        try:
            summary  = item.find('div', 'job-snippet').text.strip()
        except:
            summary = ''
    
        driver.find_element(By.).click()
        
        job = {
            'Title' : title,
            'Job_Type' : type_of_job,
            'Company' : company,
            'Salary' : salary.replace('\n', '').strip(),
            'Location' : location,
            'Date_Posted' : date_posted,
            'Current_Date' : current_date,
            'Summary' : summary,
            'Apply_link' : link
        }
        job_data.append(job)
        print('[*] Saving')

df_NY = pd.DataFrame(job_data)

['$18 - $22 an hour', 'Full-time']
[*] Saving
['Estimated $28.5K - $36.1K a year', 'Full-time']
[*] Saving
['$15 an hour', 'Full-time']
[*] Saving
['$15 an hour', 'Full-time']
[*] Saving
['$20 - $24 an hour', 'Full-time', 'Weekend availability+1']
[*] Saving
['$60,000 - $65,000 a year', 'Full-time']
[*] Saving
['$21 - $24 an hour', 'Full-time']
[*] Saving
['From $21 an hour', 'Full-time+1', 'Day shift+2']
[*] Saving
['$22 - $25 an hour', 'Full-time']
[*] Saving
['Estimated $35.2K - $44.5K a year', 'Full-time']
[*] Saving
['Estimated $26.3K - $33.4K a year']
[*] Saving
['$20 - $24 an hour', 'Full-time', 'Evening shift']
[*] Saving
['Part-time', 'Weekend availability']
[*] Saving
['Estimated $26.3K - $33.3K a year', 'Full-time']
[*] Saving
['Estimated $28.9K - $36.6K a year']
[*] Saving
['Estimated $28.1K - $35.6K a year', 'Full-time']
[*] Saving
['Up to $50 an hour', 'Full-time+1', 'Monday to Friday+5']
[*] Saving
['Estimated $53.5K - $67.7K a year', 'Full-time+1']
[*] Saving
['Estimate

['$95,000 - $125,000 a year', 'Full-time']
[*] Saving
['$20 - $28 an hour', 'Full-time+1', 'Day shift+2']
[*] Saving
['$38,480 a year', 'Part-time']
[*] Saving
['$18 - $20 an hour', 'Part-time', 'Monday to Friday+3']
[*] Saving
['$20 an hour', 'Full-time+1', 'Monday to Friday+5']
[*] Saving
['Estimated $29.1K - $36.9K a year']
[*] Saving
['$17 - $24 an hour', 'Full-time+1', '8 hour shift+1']
[*] Saving
['$15 an hour', 'Full-time+1', 'Weekend availability+2']
[*] Saving
['$17.50 - $18.00 an hour', 'Full-time', 'Monday to Friday+6']
[*] Saving
['Estimated $29.8K - $37.7K a year', 'Full-time']
[*] Saving
['$15 an hour', 'Full-time+1', 'Weekend availability+2']
[*] Saving
['$19 - $23 an hour', 'Full-time', 'Monday to Friday+2']
[*] Saving
['Estimated $28.9K - $36.6K a year', 'Full-time', 'Weekend availability+1']
[*] Saving
['$23 - $33 an hour', 'Part-time', 'Monday to Friday+4']
[*] Saving
['$15 - $18 an hour', 'Full-time', 'Weekend availability+1']
[*] Saving
['Estimated $28.9K - $36.6K 

['Estimated $40.5K - $51.3K a year']
[*] Saving
['Estimated $24.3K - $30.7K a year', 'Full-time']
[*] Saving
['Estimated $33.4K - $42.3K a year']
[*] Saving
['Estimated $30.9K - $39.2K a year']
[*] Saving
['Estimated $31K - $39.2K a year', 'Part-time']
[*] Saving
['Estimated $39.1K - $49.5K a year', 'Full-time', 'Weekend availability']
[*] Saving
['$18 - $20 an hour']
[*] Saving
['$17 an hour', 'Full-time', 'Weekend availability+2']
[*] Saving
['$18 - $22 an hour', 'Full-time']
[*] Saving
['$20 an hour', 'Full-time+1', 'Monday to Friday']
[*] Saving
['Estimated $25.5K - $32.2K a year']
[*] Saving
['$16.08 an hour', 'Part-time']
[*] Saving
['$28.86 - $38.48 an hour', 'Full-time']
[*] Saving
['$23 - $26 an hour', 'Full-time', 'Weekend availability+3']
[*] Saving
['From $15 an hour', 'Full-time', 'Weekend availability+1']
[*] Saving
['$18 an hour', 'Full-time', 'Weekend availability+1']
[*] Saving
['$22 - $28 an hour', 'Full-time']
[*] Saving
['$18.78 - $20.30 an hour', 'Full-time', 'Week

['$14 - $15 an hour', 'Part-time', 'Monday to Friday+4']
[*] Saving
['$67,000 - $75,000 a year']
[*] Saving
['$67,000 - $75,000 a year']
[*] Saving
['$15 - $20 an hour', 'Full-time']
[*] Saving
['Estimated $33.1K - $41.9K a year', 'Full-time']
[*] Saving
['$18 - $22 an hour', 'Full-time']
[*] Saving
['Full-time+1', 'Monday to Friday+4']
[*] Saving
['$15.00 - $17.50 an hour', 'Full-time+1']
[*] Saving
['From $14.13 an hour', 'Full-time', 'Weekend availability+1']
[*] Saving
['$23 - $25 an hour', 'Full-time', 'Monday to Friday+1']
[*] Saving
['$67,000 - $75,000 a year']
[*] Saving
['$97,125 a year']
[*] Saving
['$70,000 - $75,000 a year', 'Full-time', 'Monday to Friday+4']
[*] Saving
['$16.50 an hour', 'Full-time', 'Monday to Friday+8']
[*] Saving
['$70,000 - $80,000 a year', 'Full-time', 'Weekend availability+1']
[*] Saving
['Full-time']
[*] Saving
['$100 - $200 a day', 'Full-time']
[*] Saving
['From $25 an hour', 'Full-time', 'Monday to Friday+8']
[*] Saving
['Estimated $45K - $56.9K a

['$15 - $20 an hour', 'Full-time+1', 'Monday to Friday+5']
[*] Saving
['$95,000 - $110,000 a year', 'Full-time', 'Monday to Friday+8']
[*] Saving
['$800 - $1,500 a week', 'Full-time', '8 hour shift']
[*] Saving
['$58,500 - $75,000 a year', 'Full-time']
[*] Saving
['$85,000 - $95,000 a year', 'Full-time', 'Monday to Friday+5']
[*] Saving
['From $65,000 a year', 'Full-time', 'Monday to Friday+8']
[*] Saving
['$75,000 - $85,000 a year', 'Full-time', 'Day shift']
[*] Saving
['$18 - $25 an hour', 'Full-time+1']
[*] Saving
['$18 - $22 an hour', 'Full-time+1', 'Monday to Friday+5']
[*] Saving
['$80,000 - $85,000 a year', 'Full-time', 'Day shift+6']
[*] Saving
['$80,000 - $85,000 a year', 'Full-time']
[*] Saving
['$18 - $20 an hour', 'Full-time', 'Monday to Friday+2']
[*] Saving
['$750 - $900 a week', 'Full-time', 'Monday to Friday']
[*] Saving
['$57,000 - $70,000 a year', 'Full-time', 'Monday to Friday+8']
[*] Saving
['$28 - $30 an hour', 'Full-time', 'Day shift+1']
[*] Saving
['$18 an hour']

['$15 an hour', 'Part-time', '8 hour shift+1']
[*] Saving
['$15 an hour', 'Full-time+1', 'Monday to Friday+6']
[*] Saving
['$17.00 - $20.40 an hour', 'Full-time']
[*] Saving
['Estimated $27.8K - $35.3K a year', 'Full-time+1']
[*] Saving
['$17 an hour', 'Full-time', 'Weekend availability+1']
[*] Saving
['$18 - $20 an hour']
[*] Saving
['Estimated $27K - $34.2K a year', 'Full-time']
[*] Saving
['$19 - $22 an hour', 'Part-time', 'Choose your own hours']
[*] Saving
['Estimated $28.7K - $36.3K a year', 'Full-time+1']
[*] Saving
['$150 - $400 a day', 'Full-time+2', 'Monday to Friday+7']
[*] Saving
['$17 - $19 an hour']
[*] Saving
['$65,000 - $70,000 a year', 'Full-time']
[*] Saving
['$15 - $20 an hour', 'Full-time+1', 'Monday to Friday+5']
[*] Saving
['Up to $15 an hour', 'Full-time+1', 'Monday to Friday+3']
[*] Saving
['From $16.50 an hour', 'Full-time+1', 'Monday to Friday+1']
[*] Saving
['$80,000 - $100,000 a year', 'Full-time', 'Monday to Friday+4']
[*] Saving
['$58,500 - $70,000 a year'

['Estimated $46.4K - $58.8K a year']
[*] Saving
['$20 - $25 an hour', 'Full-time+1', 'Monday to Friday+5']
[*] Saving
['Estimated $31.3K - $39.6K a year']
[*] Saving
['Estimated $29.2K - $37K a year']
[*] Saving
['Up to $15 an hour', 'Full-time+1', 'Weekend availability+1']
[*] Saving
['Estimated $31.3K - $39.6K a year']
[*] Saving
['Estimated $30.4K - $38.5K a year']
[*] Saving
['$70,000 - $90,000 a year', 'Full-time', 'Monday to Friday+5']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['Estimated $31.3K - $39.6K a year']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['$80,000 a year', 'Full-time', 'Weekend availability+1']
[*] Saving
['Estimated $46.4K - $58.8K a year']
[*] Saving
['Estimated $28.8K - $36.4K a year']
[*] Saving
['Estimated $31.3K - $39.6K a year']
[*] Saving
['Estimated $31.3K - $39.6K a year']
[*] Saving
['$19 - $22 an hour', 'Full-time']
[*] Saving
['$80,000 - $85,000 a year']
[*] Saving
['Es

['Estimated $29.8K - $37.7K a year']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['$16 an hour', 'Full-time']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['$70,000 - $75,000 a year', 'Full-time']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['Estimated $31.3K - $39.6K a year']
[*] Saving
['Estimated $55.6K - $70.4K a year', 'Full-time']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['$14.13 - $20.00 an hour', 'Full-time+1', 'Monday to Friday+9']
[*] Saving
['Estimated $31.3K - $39.6K a year']
[*] Saving
['Estimated $47.7K - $60.4K a year']
[*] Saving
['Estimated $29.7K - $37.6K a year', 'Full-time+1']
[*] Saving
['Estimated $33.4K - $42.2K a year']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['$16 an hour', 'Full-time']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['$70,000 - $75,000 a year', 'Full-time']
[

['$80,000 - $85,000 a year']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['$16 an hour', 'Full-time']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['$70,000 - $75,000 a year', 'Full-time']
[*] Saving
['Estimated $31.3K - $39.6K a year']
[*] Saving
['Estimated $55.6K - $70.4K a year', 'Full-time']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['$14.13 - $20.00 an hour', 'Full-time+1', 'Monday to Friday+9']
[*] Saving
['Estimated $31.3K - $39.6K a year']
[*] Saving
['Estimated $47.7K - $60.4K a year']
[*] Saving
['Estimated $29.7K - $37.6K a year', 'Full-time+1']
[*] Saving
['Estimated $33.4K - $42.2K a year']
[*] Saving
['$80,000 - $85,000 a year']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['$16 an hour', 'Full-time']
[*] Saving
['Estimated $29.8K - $37.7K a year']
[*] Saving
['$70,000 -

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.127)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00548893+48451]
	(No symbol) [0x004DB8A1]
	(No symbol) [0x003E5058]
	(No symbol) [0x003CD073]
	(No symbol) [0x0042DEBB]
	(No symbol) [0x0043BFD3]
	(No symbol) [0x0042A0B6]
	(No symbol) [0x00407E08]
	(No symbol) [0x00408F2D]
	GetHandleVerifier [0x007A8E3A+2540266]
	GetHandleVerifier [0x007E8959+2801161]
	GetHandleVerifier [0x007E295C+2776588]
	GetHandleVerifier [0x005D2280+612144]
	(No symbol) [0x004E4F6C]
	(No symbol) [0x004E11D8]
	(No symbol) [0x004E12BB]
	(No symbol) [0x004D4857]
	BaseThreadInitThunk [0x775500C9+25]
	RtlGetAppContainerNamedObjectPath [0x77747B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77747B1E+238]


In [4]:
df = pd.DataFrame(job_data)

In [5]:
df

,Title,Job_Type,Company,Salary,Location,Date_Posted,Current_Date,Summary,Apply_link
0,Host,Full-time,La Pecora Bianca - Bryant Park,$18 - $22 an hour,"New York, NY",PostedJust posted,23-05-28,Company covers 50% of the total premium cost!\...,https://www.indeed.com//rc/clk?jk=fd14df92215f...
1,Server,Full-time,Kochi,Estimated $28.5K - $36.1K a year,"New York, NY",PostedJust posted,23-05-28,https://guide.michelin.com/us/en/new-york-stat...,https://www.indeed.com//rc/clk?jk=10452b5dd686...
2,Kitchen Server,Full-time,Eleven Madison Park,$15 an hour,"New York, NY 10010 (NoMad area)",PostedJust posted,23-05-28,Weekly education classes on various topics inc...,https://www.indeed.com//rc/clk?jk=73e1d1a97472...
3,Maitre D,Full-time,Eleven Madison Park,$15 an hour,"New York, NY 10010 (NoMad area)",PostedJust posted,23-05-28,Weekly education classes on various topics inc...,https://www.indeed.com//rc/clk?jk=d0e93b74c3ea...
4,Line Cook,Full-time,Pig & Khao,$20 - $24 an hour,"New York, NY 10002 (Lower East Side area)",PostedJust posted,23-05-28,Ensuring that all dishes are consistently prep...,https://www.indeed.com//rc/clk?jk=d27439630e9e...
...,...,...,...,...,...,...,...,...,...
1319,Assistant Manager,Full-time+1,WE CRAVE PIZZA,$14.13 - $20.00 an hour,"Hillsdale, NJ 07642",EmployerActive 3 days ago,23-05-28,"Daily responsibilities include answer phones, ...",https://www.indeed.com//company/WE-CRAVE-PIZZA...
1320,Server,Full-time+1,"036813,391 S WASHINGTON AVE,,BERGENFIELD,NJ",Estimated $31.3K - $39.6K a year,"Bergenfield, NJ 07621",PostedPosted 4 days ago,23-05-28,And you’re at least 18 years old.\nKeep in min...,https://www.indeed.com//rc/clk?jk=0a3e40ad4814...
1321,Restaurant General Manager,Full-time+1,"036802,701 STATE ROUTE 440,,JERSEY CITY,NJ",Estimated $47.7K - $60.4K a year,"Jersey City, NJ 07305",PostedPosted 4 days ago,23-05-28,To eat. To laugh. To share. That’s why people ...,https://www.indeed.com//rc/clk?jk=736f322952c6...
1322,Cook,Full-time+1,Morristown Post Acute Rehabilitation & Nursing...,Estimated $29.7K - $37.6K a year,"Morristown, NJ 07960",PostedPosted 4 days ago,23-05-28,As the Cook you will prepare meals and follow ...,https://www.indeed.com//rc/clk?jk=c1f7cb3e523d...


In [6]:
data = df.to_dict(orient="records")
db.Job_Posts.insert_many(data)

In [14]:
df["Summary"][6]

"Beverage-wise, K'Far will offer a full coffee drink menu and cocktails with nods to Isreal.\nOpen schedule availability is preferred, and open minds are a must."